In [ ]:
%cd ..

In [ ]:
import json
import os
import pickle


In [ ]:
from apperception.database import database
from apperception.world import empty_world
from apperception.utils import F, join, import_pickle
from apperception.predicate import camera, objects, lit
database.connection

In [ ]:
from optimized_ingestion.camera_config import camera_config
from optimized_ingestion.payload import Payload
from optimized_ingestion.pipeline import Pipeline
from optimized_ingestion.stages.in_view import InView
from optimized_ingestion.stages.decode_frame.parallel_decode_frame import ParallelDecodeFrame
from optimized_ingestion.stages.decode_frame.decode_frame import DecodeFrame
from optimized_ingestion.stages.detection_2d.yolo_detection import YoloDetection
from optimized_ingestion.stages.detection_2d.object_type_filter import ObjectTypeFilter
# from optimized_ingestion.stages.filter_car_facing_sideway import FilterCarFacingSideway
from optimized_ingestion.stages.detection_estimation import DetectionEstimation
from optimized_ingestion.stages.tracking_2d.strongsort import StrongSORT
from optimized_ingestion.stages.detection_3d.from_2d_and_road import From2DAndRoad
from optimized_ingestion.video import Video

In [ ]:
BOSTON_VIDEOS = [
#     "scene-0757-CAM_FRONT",
    # "scene-0103-CAM_FRONT",
#     "scene-0553-CAM_FRONT",
    # "scene-0665-CAM_FRONT",
    "scene-0655-CAM_FRONT",
#     "scene-0655-CAM_FRONT_RIGHT",
#     "scene-0655-CAM_BACK_RIGHT",
#     "scene-0553-CAM_FRONT_LEFT"
#     "scene-0103-CAM_FRONT"
]

NUSCENES_PROCESSED_DATA = "NUSCENES_PROCESSED_DATA"

In [ ]:
name = 'ScenicWorld'
world = empty_world(name=name)

obj1 = objects[0]
obj2 = objects[1]
cam = camera

world = world.filter(
    (obj1.id != obj2.id) &
    F.like(obj1.type, 'car') &
    F.like(obj2.type, 'car') &
    F.angle_between(F.facing_relative(cam.ego, F.road_direction(cam.ego)), -15, 15) &
    (F.distance(cam.ego, obj1.trans@cam.time) < 50) &
    (F.view_angle(obj1.trans@cam.time, cam.ego) < 70 / 2.0) &
    (F.distance(cam.ego, obj2.trans@cam.time) < 50) &
    (F.view_angle(obj2.trans@cam.time, cam.ego) < 70 / 2.0) &
    F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, cam.ego), 50, 135) &
    F.angle_between(F.facing_relative(obj2.trans@cam.time, cam.ego), -135, -50) &
    (F.min_distance(cam.ego, 'intersection') < 10) &
    F.angle_between(F.facing_relative(obj1.trans@cam.time, obj2.trans@cam.time), 100, -100)
)


In [ ]:
# F.contains_all('intersection', [obj1.trans, obj2.trans]@cam.time) => lambda x: x.road_type == 'intersection'
# F.min_distance(cam.ego, 'intersection') < 10 => InView(distance=10, segment_type='intersection')
# F.like(obj1.type, 'vehicle%') => ObjectTypeFilter stage
# F.distance(cam.ego, obj1.trans@cam.time) < 50 =>
    # compute_distance(x.car_loc3d, x.ego_config.ego_translation) < 50
from apperception.predicate import (Visitor, PredicateNode, CallNode,
                                    CompOpNode, TableAttrNode, BinOpNode, LiteralNode)

def in_view(pipeline, param):
    pipeline.stages.insert(0, InView(**param))
    
def object_type(pipeline, param):
    for i in range(len(pipeline.stages)):
        if isinstance(pipeline.stages[i], YoloDetection):
            pipeline.stages.insert(i+1, ObjectTypeFilter(param))
    
def road_type(pipeline, param):
    for s in pipeline.stages:
        if isinstance(s, DetectionEstimation):
            s.filter(lambda x: x.road_type == param)
            
def distance_to_ego(pipeline, param):
    for s in pipeline.stages:
        if isinstance(s, DetectionEstimation):
            s.filter(lambda x: compute_distance(
                x.car_loc3d, x.ego_config.ego_translation) < param)
            
            
ALL_MAPPING_RULES = {
    'in_view': {'condition': lambda x: (isinstance(x, CompOpNode) and
                                        isinstance(x.left, CallNode) and
                                        isinstance(x.right, LiteralNode) and
                                        x.left._fn[0].__name__ == 'fn' and
                                        isinstance(x.left.params[0], TableAttrNode) and
                                        x.left.params[0].name == 'egoTranslation' and
                                        isinstance(x.left.params[1], LiteralNode)),
                'param': lambda x: dict(segment_type=x.left.params[1].value, distance=x.right.value),
                'pipeline': in_view},
    'object_type': {'condition': lambda x: (isinstance(x, CallNode) and
                                            x._fn[0].__name__ == 'like' and
                                            x.params[0].name == 'objectType'),
                    'param': lambda x: [x.params[1].value],
                    'pipeline': object_type},
    'road_type': {'condition': lambda x: (isinstance(x, CallNode) and
                                         x._fn[0].__name__ == 'contains_all'),
                  'param': lambda x: x.params[0].value,
                  'pipeline': road_type},
    'distance_to_ego': {'condition': lambda x: (isinstance(x, CompOpNode) and
                                                isinstance(x.left, CallNode) and
                                                isinstance(x.right, LiteralNode) and
                                                x.left._fn[0].__name__ == 'fn' and
                                                isinstance(x.left.params[0], TableAttrNode) and
                                                x.left.params[0].name == 'egoTranslation' and
                                                isinstance(x.left.params[1], BinOpNode)),
                       'param': lambda x: x.right.value,
                       'pipeline': distance_to_ego}
}

def pipeline_rule(pipeline, node):
    for key, rule in ALL_MAPPING_RULES.items():
        if rule['condition'](node):
            param = rule['param'](node)
            rule['pipeline'](pipeline, param)
                
class PipelineConstructor(Visitor[PredicateNode]):
    
    def add_pipeline(self, pipeline):
        self.pipeline = pipeline
        return self

    def visit_CompOpNode(self, node: "CompOpNode"):
        assert self.pipeline
        pipeline_rule(self.pipeline, node)
        self(node.left)
        self(node.right)

    def visit_CallNode(self, node: "CallNode"):
        assert self.pipeline
        pipeline_rule(self.pipeline, node)
        for p in node.params:
            self(p)


In [ ]:
def construct_pipeline(world):
    pipeline = Pipeline()
    pipeline.add_filter(filter=ParallelDecodeFrame())
    pipeline.add_filter(filter=YoloDetection())

    pipeline.add_filter(filter=From2DAndRoad())
    pipeline.add_filter(filter=DetectionEstimation())  # 5 Frame p Second
    pipeline.add_filter(filter=StrongSORT())  # 2 Frame p Second
    PipelineConstructor().add_pipeline(pipeline)(world.kwargs['predicate'])
    return pipeline

In [ ]:
def associate_detection_info(tracking_result, detection_info_meta):
    for detection_info in detection_info_meta[tracking_result.frame_idx]:
        if detection_info.detection_id == tracking_result.detection_id:
            return detection_info
        
def get_tracks(sortmeta, detection_estimation_meta):
    trajectories = {}
    for frame in sortmeta:
        for obj_id, tracking_result in frame.items():
            if obj_id not in trajectories:
                trajectories[obj_id] = []
            associated_detection_info = associate_detection_info(
                tracking_result, detection_estimation_meta)
            trajectories[obj_id].append((tracking_result, associated_detection_info))

    for trajectory in trajectories.values():
        last = len(trajectory) - 1
        for i, t in enumerate(trajectory):
            if i > 0:
                t[0].prev = trajectory[i - 1][0]
            if i < last:
                t[0].next = trajectory[i + 1][0]
    return trajectories

In [ ]:
from typing import List, Tuple
def insert_trajectory(
    database,
    item_id: str,
    camera_id: str,
    object_type: str,
    postgres_timestamps: List[str],
    pairs: List[Tuple[float, float, float]],
    itemHeading_list: List[int],
    translation_list: List[Tuple[float, float, float]],
    road_types: List[str],
    roadpolygon_list: List[List[Tuple[float, float]]]
):
    ### Save camera config
    PICKLE_DATA_PATH = '/data/apperception-data/processed/nuscenes/full-dataset-v1.0/Mini/videos/boston-seaport'
    import_pickle(database, PICKLE_DATA_PATH)
    traj_centroids = []
    translations = []
    itemHeadings = []
    roadTypes = []
    roadPolygons = []
    prevTimestamp = None
    for timestamp, current_point, curItemHeading, current_trans, cur_road_type, cur_roadpolygon in zip(
        postgres_timestamps, pairs, itemHeading_list, translation_list, road_types, roadpolygon_list
    ):
        if prevTimestamp == timestamp:
            continue
        prevTimestamp = timestamp

        # Construct trajectory
        traj_centroids.append(f"POINT Z ({join(current_point, ' ')})@{timestamp}")
        translations.append(f"POINT Z ({join(current_trans, ' ')})@{timestamp}")
        itemHeadings.append(f"{curItemHeading}@{timestamp}")
        roadTypes.append(f"{cur_road_type}@{timestamp}")
#         polygon_point = ', '.join(join(cur_point, ' ') for cur_point in list(
#             zip(*cur_roadpolygon.exterior.coords.xy)))
#         roadPolygons.append(f"Polygon (({polygon_point}))@{timestamp}")

    # Insert the item_trajectory separately
    insert_trajectory = f"""
    INSERT INTO Item_General_Trajectory (itemId, cameraId, objectType, roadTypes, trajCentroids,
    translations, itemHeadings)
    VALUES (
        '{item_id}',
        '{camera_id}',
        '{object_type}',
        ttext '{{[{', '.join(roadTypes)}]}}',
        tgeompoint '{{[{', '.join(traj_centroids)}]}}',
        tgeompoint '{{[{', '.join(translations)}]}}',
        tfloat '{{[{', '.join(itemHeadings)}]}}'
    );
    """

    database.execute(insert_trajectory)

In [ ]:
def format_trajectory(obj_id, track):
    timestamps: List[str] = []
    pairs: List[Tuple[float, float, float]] = []
    itemHeadings: List[int] = []
    translations: List[Tuple[float, float, float]] = []
    road_types: List[str] = []
    roadpolygons: List[List[Tuple[float, float]]] = []

    for tracking_result_2d, detection_info in track:
        if detection_info:
            camera_id = detection_info.ego_config.camera_id
            object_type = tracking_result_2d.object_type
            timestamps.append(detection_info.timestamp)
            pairs.append(detection_info.car_loc3d)
            itemHeadings.append(detection_info.segment_heading)
            translations.append(detection_info.ego_config.ego_translation)
            road_types.append(detection_info.road_type)
            roadpolygons.append(detection_info.road_polygon_info.polygon)
    print([obj_id, camera_id, object_type, timestamps, pairs,
            itemHeadings, translations, road_types, roadpolygons])
    return [obj_id, camera_id, object_type, timestamps, pairs,
            itemHeadings, translations, road_types, roadpolygons]

In [ ]:
def process_pipeline(frames, pipeline, insert=False):
    output = pipeline.run(Payload(frames)).__dict__
    metadata = output['metadata']
    kept_fn = [i for i, val in enumerate(output['keep']) if val]

    detection_estimation_meta = metadata['DetectionEstimation']
    sortmeta = metadata['Tracking2D.StrongSORT']
    tracks = get_tracks(sortmeta, detection_estimation_meta)
    for obj_id, track in tracks.items():
        if insert:
            insert_trajectory(database, *format_trajectory(obj_id, track))


In [ ]:
def preprocess(world):
    pipeline = construct_pipeline(world)
    if NUSCENES_PROCESSED_DATA in os.environ:
        DATA_DIR = os.environ[NUSCENES_PROCESSED_DATA]
    else:
        DATA_DIR = "/work/apperception/data/nuScenes/full-dataset-v1.0/Mini"
    
    video_dir = os.path.join(DATA_DIR, 'videos')
    with open(os.path.join(video_dir, "frames.pkl"), "rb") as f:
        videos = pickle.load(f)

    num_video = 0
    for name, video in videos.items():
        if name not in BOSTON_VIDEOS:
            continue
    #     if not name.endswith('CAM_FRONT'):
    #         continue
    #     if 'CAM_FRONT' not in name:
    #         continue

        print(name, '--------------------------------------------------------------------------------')
        frames = Video(
            os.path.join(video_dir, video["filename"]),
            [camera_config(*f, 0) for f in video["frames"]],
            video["start"],
        )
        process_pipeline(frames, pipeline)
        

In [ ]:
preprocess(world)

In [ ]:
id_time_camId_filename = world.get_id_time_camId_filename(2)

In [ ]:
id_time_camId_filename